# Running MYSQL in Python

In [4]:
import pymysql
from sqlalchemy import create_engine
from getpass import getpass

In [5]:
username='root'
server='localhost'
database='ironhack'
password=getpass()

engine=create_engine(f'mysql+pymysql://{username}:{password}@{server}/vivino')

········


In [6]:
'''
#what food goes with different wines?


select * from meat_wine mw 
left join meat_info mi 
on mw.food_id=mi.id;'''

'\n#what food goes with different wines?\n\n\nselect * from meat_wine mw \nleft join meat_info mi \non mw.food_id=mi.id;'

In [7]:
#option 1
pd.read_sql_query('''
#what food goes with different wines?


select * from meat_wine mw 
left join meat_info mi 
on mw.food_id=mi.id;''', engine)

,wine_id,food_id,id,name,seo_name
0,1163903,4,4,Bœuf,beef
1,1163903,8,8,Agneau,lamb
2,1163903,9,9,Veau,veal
3,1163903,20,20,Volaille,poultry
4,1873,4,4,Bœuf,beef
...,...,...,...,...,...
3141,1143692,11,11,"Gibier (cerf, chevreuil)",game
3142,1622868,4,4,Bœuf,beef
3143,1622868,8,8,Agneau,lamb
3144,1622868,11,11,"Gibier (cerf, chevreuil)",game


In [8]:
#option 2
mw=pd.read_sql_table('meat_wine', engine)
mi=pd.read_sql_table('meat_info', engine)

In [10]:
mw.merge(mi, left_on='food_id', right_on='id', how='inner')[['wine_id','name']]

,wine_id,name
0,1163903,Bœuf
1,1873,Bœuf
2,4848,Bœuf
3,7773,Bœuf
4,4113,Bœuf
...,...,...
3141,15098,Apéritif
3142,1230950,Apéritif
3143,1210421,Apéritif
3144,1230960,Apéritif


In [11]:
'''select wine_id, group_concat(distinct name)    #this way you aggregate the names into a single value with comas
from meat_wine mw          #distinct so that there will be no repetition
inner join meat_info mi 
on mw.food_id=mi.id
group by wine_id;'''

'select wine_id, group_concat(distinct name)    #this way you aggregate the names into a single value with comas\nfrom meat_wine mw          #distinct so that there will be no repetition\ninner join meat_info mi \non mw.food_id=mi.id\ngroup by wine_id;'

In [13]:
joined_table=mw.merge(mi, left_on='food_id', right_on='id', how='inner')

In [17]:
joined_table.groupby('wine_id').name.apply(list)  #you apply list to column name

wine_id
1241                                [Bœuf, Agneau, Volaille]
1244       [Bœuf, Agneau, Volaille, Gibier (cerf, chevreu...
1245       [Bœuf, Agneau, Volaille, Gibier (cerf, chevreu...
1249       [Volaille, Végétarien, Poisson gras (saumon, t...
1251                                [Bœuf, Agneau, Volaille]
                                 ...                        
6514155                                [Bœuf, Agneau, Pâtes]
6689232     [Bœuf, Veau, Volaille, Gibier (cerf, chevreuil)]
7199473           [Crustacés, Végétarien, Fromage de chèvre]
7311113                       [Bœuf, Agneau, Veau, Volaille]
7334207                      [Bœuf, Agneau, Volaille, Pâtes]
Name: name, Length: 792, dtype: object

In [19]:
joined_table.groupby('wine_id').name.apply(list).reset_index()

,wine_id,name
0,1241,"[Bœuf, Agneau, Volaille]"
1,1244,"[Bœuf, Agneau, Volaille, Gibier (cerf, chevreu..."
2,1245,"[Bœuf, Agneau, Volaille, Gibier (cerf, chevreu..."
3,1249,"[Volaille, Végétarien, Poisson gras (saumon, t..."
4,1251,"[Bœuf, Agneau, Volaille]"
...,...,...
787,6514155,"[Bœuf, Agneau, Pâtes]"
788,6689232,"[Bœuf, Veau, Volaille, Gibier (cerf, chevreuil)]"
789,7199473,"[Crustacés, Végétarien, Fromage de chèvre]"
790,7311113,"[Bœuf, Agneau, Veau, Volaille]"


In [20]:
joined_table.groupby('wine_id').name.apply(lambda x:','.join(set(x))).reset_index() #doing set is giving distinct 

,wine_id,name
0,1241,"Agneau,Volaille,Bœuf"
1,1244,"Agneau,Volaille,Bœuf,Gibier (cerf, chevreuil)"
2,1245,"Agneau,Volaille,Bœuf,Gibier (cerf, chevreuil)"
3,1249,"Végétarien,Porc,Volaille,Poisson gras (saumon,..."
4,1251,"Agneau,Volaille,Bœuf"
...,...,...
787,6514155,"Agneau,Bœuf,Pâtes"
788,6689232,"Volaille,Bœuf,Gibier (cerf, chevreuil),Veau"
789,7199473,"Végétarien,Fromage de chèvre,Crustacés"
790,7311113,"Agneau,Volaille,Bœuf,Veau"


In [ ]:
'''select wine_id, group_concat(distinct name), count(distinct name)
from meat_wine mw          
inner join meat_info mi 
on mw.food_id=mi.id
group by wine_id
order by 3 DESC 
limit 10;'''

In [22]:
joined_table.groupby('wine_id').name.agg(['nunique',lambda x:','.join(set(x))])

#aggg is used when you apply nunique values

,nunique,<lambda_0>
wine_id,,
1241,3,"Agneau,Volaille,Bœuf"
1244,4,"Agneau,Volaille,Bœuf,Gibier (cerf, chevreuil)"
1245,4,"Agneau,Volaille,Bœuf,Gibier (cerf, chevreuil)"
1249,4,"Végétarien,Porc,Volaille,Poisson gras (saumon,..."
1251,3,"Agneau,Volaille,Bœuf"
...,...,...
6514155,3,"Agneau,Bœuf,Pâtes"
6689232,4,"Volaille,Bœuf,Gibier (cerf, chevreuil),Veau"
7199473,3,"Végétarien,Fromage de chèvre,Crustacés"


In [23]:
joined_table.name.nunique()

21

In [ ]:
'''select type, count(distinct wine_id) from wine
group by type;
'''

In [24]:
wine=pd.read_sql_table('wine', engine)

In [25]:
wine.type.value_counts()

Red      3392
White     208
Name: type, dtype: int64

In [27]:
wine.groupby('wine_id').type.unique()

wine_id
1241         [Red]
1244         [Red]
1245         [Red]
1249       [White]
1251         [Red]
            ...   
6514155      [Red]
6689232      [Red]
7199473    [White]
7311113      [Red]
7334207      [Red]
Name: type, Length: 794, dtype: object

In [28]:
wine.groupby('wine_id').type.unique().apply(lambda x : x[0]).value_counts()

Red      690
White    104
Name: type, dtype: int64

In [29]:
#in this case python is shorter

In [31]:
wine.drop_duplicates(subset='wine_id').type.value_counts()

Red      690
White    104
Name: type, dtype: int64

In [32]:
wine.groupby('type').wine_id.nunique()

type
Red      690
White    104
Name: wine_id, dtype: int64

In [ ]:
'''#how many data points do we have per country
select country, count(distinct wine_id)
from wine 
group by country 
order by 2 desc;'''

In [33]:
wine.groupby('country').wine_id.nunique().sort_values(ascending=False)

country
France              470
Italie              166
Espagne              64
États-Unis           27
Chili                15
Australie            15
Argentine            13
Nouvelle-Zélande     10
Allemagne             5
Liban                 4
Afrique du Sud        3
Portugal              1
Name: wine_id, dtype: int64

In [34]:
wine[wine.country.isna()]

,vintage_id,name,certified_biodynamic,wine_id,wine_name,type,is_natural,region_id,country,winery_id,...,vintage_rank_percentile_country,vintage_rank_percentile_region,vintage_rank_percentile_winery,vintage_rank_percentile_global,contains_added_sulfites,contains_milk_allergens,contains_egg_allergens,closure,year,median_price
1911,145505353,Ao Yun (敖云葡萄园) Shangri-La (敖云) 2014,NaN,4383995,Shangri-La (敖云),Red,0,NaN,None,231611,...,0.23,6.49,41.67,4.35,NaN,NaN,NaN,None,2014.0,306.67


In [36]:
#replacing Null by China
wine.loc[wine.country.isna(), 'country']='China' 

In [37]:
wine[wine.country.isna()]

,vintage_id,name,certified_biodynamic,wine_id,wine_name,type,is_natural,region_id,country,winery_id,...,vintage_rank_percentile_country,vintage_rank_percentile_region,vintage_rank_percentile_winery,vintage_rank_percentile_global,contains_added_sulfites,contains_milk_allergens,contains_egg_allergens,closure,year,median_price


In [ ]:
'''select w.country,round(avg(price),2) as price, round(max(price),2) as maxprice, round(min(price),2) as minprice
from prices p
inner join vintages v on p.vintage_id=v.id 
inner join wine w on v.wine_id=w.wine_id
group by w.country
order by 4 desc;'''

In [38]:
prices=pd.read_sql_table('prices', engine)
vintages=pd.read_sql_table('vintages', engine)

In [40]:
# right on because the keys are different 
#on only on the second one because the keys are the same

joined_table=prices.merge(vintages, left_on='vintage_id', right_on='id').merge(wine, on='wine_id')

In [41]:
#using aggregate
joined_table[['country','price']].groupby('country').price.agg(['max','min','mean'])

,max,min,mean
country,,,
Afrique du Sud,22.90,9.00,16.790909
Allemagne,47.00,10.12,20.399853
Argentine,189.00,8.20,67.548482
Australie,165.00,11.65,89.094975
Chili,424.80,8.65,182.724659
China,330.00,274.00,295.776667
Espagne,290.00,3.75,43.492594
France,820.00,5.50,106.138083
Italie,549.00,6.20,83.929514


In [42]:
joined_table[['country','price']].groupby('country').price.describe()

,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Afrique du Sud,11.0,16.790909,5.129026,9.00,13.4500,15.90,22.1500,22.90
Allemagne,19.0,20.399853,9.228751,10.12,13.2500,18.00,24.9000,47.00
Argentine,130.0,67.548482,41.285593,8.20,20.2375,79.00,89.0000,189.00
Australie,203.0,89.094975,46.787269,11.65,35.7750,109.00,125.0000,165.00
Chili,440.0,182.724659,85.386740,8.65,139.0000,159.00,194.0800,424.80
China,3.0,295.776667,30.003160,274.00,278.6650,283.33,306.6650,330.00
Espagne,1552.0,43.492594,36.601053,3.75,24.1700,36.90,45.0000,290.00
France,167079.0,106.138083,70.839739,5.50,53.9500,93.50,140.0000,820.00
Italie,2536.0,83.929514,77.853125,6.20,26.6700,53.42,119.1700,549.00


In [44]:
#apply round to the whole dataset
joined_table[['country','price']].groupby('country').price.describe().round(2)

,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Afrique du Sud,11.0,16.79,5.13,9.00,13.45,15.90,22.15,22.90
Allemagne,19.0,20.40,9.23,10.12,13.25,18.00,24.90,47.00
Argentine,130.0,67.55,41.29,8.20,20.24,79.00,89.00,189.00
Australie,203.0,89.09,46.79,11.65,35.78,109.00,125.00,165.00
Chili,440.0,182.72,85.39,8.65,139.00,159.00,194.08,424.80
China,3.0,295.78,30.00,274.00,278.66,283.33,306.66,330.00
Espagne,1552.0,43.49,36.60,3.75,24.17,36.90,45.00,290.00
France,167079.0,106.14,70.84,5.50,53.95,93.50,140.00,820.00
Italie,2536.0,83.93,77.85,6.20,26.67,53.42,119.17,549.00
